## Imports
Execute the cell below to import all packages needed.

In [1]:
# System functionality
import sys
sys.path.append('..')

import glob
import os
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import graphviz
import math as math
import json
import numpy
import cv2
import scipy.stats as stats
import matplotlib.image as mpimg
import sklearn.feature_selection as feature_selection
from skimage import data, io, filters
from numpy import linspace
from numpy.core.umath import pi
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from emoji_extractor.extract import Extractor
from emojipedia import Emojipedia
from skimage import io
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2_contingency
from scipy.interpolate import spline

## Load csv into pandas dataframe

In [2]:
path__tweets_csv = '/home/stelios/Desktop/Honours Project/Samples/tweet_data/tweets_nyc_all.csv'
path_users_csv = '/home/stelios/Desktop/Honours Project/Samples/user_data/nyc.csv'

nyc_join_df = pd.read_csv(path__tweets_csv,index_col=None, header=0, low_memory = False).applymap(str).join(pd.read_csv(path_users_csv,index_col=None, header=0, low_memory = False).applymap(str).set_index('user_id'), on='user_id')

In [3]:
nyc_join_df.columns

Index(['user_id', 'user_name', 'user_screen_name', 'user_statuses_count',
       'tweet_id', 'hashtags', 'is_quote_status', 'text', 'created_at',
       'source', 'in_reply_to_screen_name', '_unit_id', '_golden',
       '_unit_state', '_trusted_judgments', '_last_judgment_at', 'ethnicity',
       'ethnicity:confidence', 'gender', 'gender:confidence',
       'reason_not_valid', 'reason_not_valid:confidence', 'valid_photo',
       'valid_photo:confidence',
       'which_icon_most_closely_matches_the_persons_skin_tone',
       'which_icon_most_closely_matches_the_persons_skin_tone:confidence',
       'orig__golden', 'ethnicity_gold', 'ethnicity_gold_reason',
       'gender_gold', 'gender_gold_reason', 'image_url',
       'reason_not_valid_gold', 'reason_not_valid_gold_reason',
       'valid_photo_gold', 'valid_photo_gold_reason',
       'which_icon_most_closely_matches_the_persons_skin_tone_gold',
       'which_icon_most_closely_matches_the_persons_skin_tone_gold_reason'],
      dtype='ob

In [4]:
# Gia kathe user:
# 1) Number of tweets 2) Number of tweets with emojis

# For all users (1) for all tweets, (2) for only emoji tweets:
# 1) Min number of tweets , 2) Max number of tweets, 3) Average number of tweets

In [5]:
# Load users
nyc_users_df = pd.read_csv(path_users_csv,index_col=None, header=0, low_memory = False).applymap(str)

In [6]:
# Fill pandas dataframe 
emojis_count_create_df = pd.DataFrame(columns = ['User','Tweets','Tweets_with_Emoji','Gender','Race'])
emojis_count_create_df['User'] = nyc_users_df['user_id']
emojis_count_create_df['Gender']= nyc_users_df['gender']
emojis_count_create_df['Race']= nyc_users_df['ethnicity']
emojis_count_create_df

,User,Tweets,Tweets_with_Emoji,Gender,Race
0,374293726,NaN,NaN,male,white
1,895478601893650432,NaN,NaN,female,white
2,832635277,NaN,NaN,nan,nan
3,948965128015437824,NaN,NaN,nan,nan
4,4168370177,NaN,NaN,male,nan
5,839572840995225601,NaN,NaN,female,white
6,512770910,NaN,NaN,nan,nan
7,754904544245608448,NaN,NaN,nan,nan
8,282168717,NaN,NaN,nan,nan
9,28663989,NaN,NaN,female,white


In [7]:
extract = Extractor()
# Add the frequencies of tweets by User
for enum,user in enumerate (nyc_users_df['user_id']):
    #print(enum)
    tweet_emoji_count=0
    emojis_count_create_df['Tweets'][enum] = nyc_join_df.loc[nyc_join_df['user_id'] == user, 'text'].count()
    for tweet in nyc_join_df.loc[nyc_join_df['user_id'] == user, 'text']:
        if(extract.detect_emoji(tweet)==True):
            tweet_emoji_count=tweet_emoji_count+1
    emojis_count_create_df['Tweets_with_Emoji'][enum] = tweet_emoji_count

In [8]:
# Store table
emojis_count_create_df.to_csv("/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/nyc_per_user_tweet_counts.csv",encoding='utf-8', index=False)